In [ ]:
import pandas as pd

from pathlib import Path
Path("../data/main").mkdir(parents=True, exist_ok=True)

In [ ]:
energy_macros=[
    'FAT',
    'FAPU',
    'FASAT',
    'CHOAVL',
    'PROT',
    'FIBC',
]

In [ ]:
aromi_records = pd.read_excel(
    '../data/RAW/aromi_product_recipe.xlsx', 
    sheet_name='Tutkimusraportti',
    usecols=[
        'Tunnus',
        'Reseptin/tuotteen tunnus',
        'ID',
        'NACL'
    ] + energy_macros    
)

In [ ]:
classes =  pd.read_excel(
    '../data/public/recipe_class.xlsx'
)

In [ ]:
classes['main_class'] = classes['class'].apply(lambda x: str(x)[:-2].zfill(2))
classes['subclass'] = classes['class'].apply(lambda x: str(abs(x%100)).zfill(2))

In [ ]:
food_records = aromi_records.merge(
    classes,
    left_on='Reseptin/tuotteen tunnus', 
    right_on='recipe_id',
    how='left'
)

In [ ]:
class_missing = food_records['Reseptin/tuotteen tunnus'] != food_records.recipe_id
food_records[class_missing]

In [ ]:
intake_per_person_per_subclass = food_records.groupby(['Tunnus','class']).sum()

In [ ]:
intake_per_person_per_main_class = food_records.drop(columns=['class']).groupby(['Tunnus','main_class']).sum()

In [ ]:
with pd.ExcelWriter('../data/main/intakes_simple_per_recipe.xlsx') as writer:
    food_records.to_excel(
        writer, 
        sheet_name='food_records',
        index=False
    )
    
    intake_per_person_per_subclass.to_excel(
        writer, 
        sheet_name='per_subclass',
    )

    intake_per_person_per_main_class.to_excel(
        writer, 
        sheet_name='per_main_class',
    )
